***
# Milk Run Vehicle  Routing Problem with Time window
***

## Problem Desription

Vehicle routing problem in an automobile industries set up can be described as an ecosystem in which there exists supply and demand.

The main parties involved are car manufacturer, car parts suppliers, and a shipping or logistics companies that has a fleet of vehicles. 

The shipping companies use multiple vehicles to take goods from car parts suppliers. 

The location of each supplier, volume of cargo and pickup time window and per vehicle load volume and load capacity is constant. 

In order to make sure that the total cost is minimum, it is necessary to make reasonable arrangements of the vehicles travel path and traveling time while shipping the car parts.

The vehicle routing design of the shipping companies routine need to meet the following assumptions:
    
-  	The goods to be shipped are one or more kind of goods that is known.
-  	The location of each user (supplier) and quantity of supply is known.
-  	The load distances from the logistics center to the various suppliers and the distance between each supplier is known
-  	The distribution center has enough resources and capacity.
-  	Fixed cost of each vehicle are the same, however, the variable cost is proportional to the distance, therefore the greater the distance, the higher the cost, so the objective function directly expressed as to minimize the total distance.
-  	The average speed of each car are same, this problem doesn’t consider the effect of traffic.
-  	The vehicle scheduling problem is under identical state, meaning the pickup vehicles of the logistics
-  	company are identical.
-  	The loading dynamics is the same. The loading time allocation is 20 min per supplier.
-  	Car parts obtained from different suppliers can be shipped by the same vehicle.
-  	Car parts suppliers are ready with parts to be shipped within a prescribed time window.

Moreover, the following conditions must be fulfilled:
    
-  	All suppliers demand must be fullfiled.
-  	Supplier’s orders can only be shipped by vehicles from the logistic companies. 
-  	There is a certain limit of each pickup vehicle load (300 units maximum), the vehicle cannot be overloaded.
-  	 Vehicles must set off from car manufacturers, and in the end they will be back in the car factory.
-  	Every car should take goods from suppliers within the prescribed time window, otherwise it is invalid.
-  	There is a predetermined limit for each car total run time and distance coverage every day.


---
## Model Formulation

### Sets and Indices
$i, j \in \text{Suppliers} = \{0,1,2, ..,18 \}$: Indices and set of suppliers. The depot index number is 0.

$\text{Edges}= \{(i,j) \in Suppliers \times Suppliers \}$: Set of allowed Edges.

$k \in K = \{1,2, 3 \}  $: Vehicles available $k$.

$S_k \in S  $: Subtour in route of vehicle $k$.

$G = (\text{Suppliers} , \text{Edges})$: A graph where the set $\text{Suppliers}$ defines the set of nodes and the set $\text{Edges}$ defines the set of edges. 

### Parameters 

$d_{i, j} \in \mathbb{R}^+$: Distance from supplier $i$ to supplier $j$, for all $(i, j) \in \text{Edges}$. 

Notice that the distance from supplier $i$ to supplier $j$ is the same as the distance from supplier $j$ to upplier $i$, i.e. $d_{i, j} = d_{j, i}$.

$C \in \mathbb{R}^+$: The capacity of the vehicle.

$q_i \in \mathbb{Z}^+$: Units to be shipped by supplier $ i \in Suppliers$.

$t_{i, j} \in \mathbb{R}^+$: Travel time from $i$ to $j$ $ (i,j) \in Suppliers$.

$T_i \in \mathbb{R}^+$: The load time allocated for $ i \in Suppliers$.

$ET_i \in \mathbb{R}^+$: The earliest pick-up time.

$LT_i \in \mathbb{R}^+$: The lateest pick-up time.

### Decision Variables
$x_{i, j, k} \in \{0, 1\}$: This variable is equal to 1, if the vehicle drives from supplier $i$ to supplier $j$. Otherwise, the decision variable is equal to zero.

$y_{i, k} \in \{0, 1\}$: This variable is equal to 1, if vehicle $k \in K$ gets goods from supplier $i \in Suppliers$.

$Arr_i \in \mathbb{R}^+$: The arrival time of vehicle $k$ at supplier $i \in Suppliers $

### Objective Function
- **Shortest Route**. Minimize the total distance covered  of all vehicles .$M$ is associated large penalty cost for vehicle pick-up delays not within Supplier pick-up time window $[Et_i, LT_i]$

\begin{equation}
\text{Min} \quad Z = \sum_{i \in Suppliers}\sum_{j \in Suppliers}\sum_{k \in K} d_{i, j}\cdot x_{i, j, k}  + M\cdot\sum_{i \in Suppliers}Max (ET_i - Arr_i, 0) + M\cdot\sum_{i \in Suppliers} Max(Arr_i - LT_i, 0) 
\tag{0}
\end{equation}

### Constraints 
- **Symmetry Constraints**. For each edge $(i,j)$, ensure that the supplier $i$ and $j$ are connected, if the former is visited immediately before or after visiting the latter.

\begin{equation}
x_{i, j, k} = x_{j, i, k} \quad \forall k \in K, \; (i, j) \in Edges
\tag{1}
\end{equation}

- **Entering and leaving a Supplier**. For each supplier $i$, ensure that this supplier is connected to two other suppliers. 

\begin{equation}
\sum_{j: (i,j) \in \text{Edges}} x_{i,j,k} = 2 \cdot y_{i, k} \quad \forall  i \in Suppliers, \; k \in K 
\tag{2}
\end{equation}


- **Vehicle capacity**. The tanker has limited capacity.
\begin{equation}
\sum_{i \in \text{Suppliers}} q_{i} \cdot y_{i,k} \leq C -\sum_{i \in Suppliers} q_{i} \quad \forall  k \in K 
\tag{4}
\end{equation}

- **Suppliers visited**. Limit on pick-up by only one vehicle.

\begin{equation}
\sum_{k \in \text{K}} y_{k,i}  = 1 \quad \forall  i \in \text{Suppliers}
\tag{5}
\end{equation}

- **Subtour elimination**. These constraints ensure that for each route there is no cycle. 

\begin{equation}
\sum_{(i,j) \in S_k}x_{i,j,k} \leq |S_k|-1 \quad \forall  k \in K, \;   S_k \in S
\tag{6}
\end{equation}

Where the subset $S$ is the set of Suppliers visited by the tour, and this tour is defined by the values of the decision variables in the LHS of the inequality.

- **Time chronology**. These constraints ensure that vehicle $k$ reaches the vendor $j$ from vendor $i$ in no time less then $Arr_i + T_i +t_{i,j}$

\begin{equation}
Arr_i + T_i +t_{i,j}-M(1-x_{i,j,k}) \leq  Arr_j \forall  (i,j) \in Suppliers, \;   k \in K
\tag{7}
\end{equation}

## Python Implementation

We import openrouteservice Module, folium and other Python libraries.

In [1]:
import folium
from folium.plugins import BeautifyIcon
import pandas as pd
import openrouteservice as ors

In [2]:
# First define the map centered around Tempelhof in Berlin
m = folium.Map(location=[52.46806664515245, 13.37068018429439], tiles='cartodbpositron', zoom_start=6)
m

### Input data  
We define all the input data for the model.

In [3]:
# Next load the delivery locations from CSV file at ../milkrun.csv
# ID, Lat, Lon, Open_From, Open_To, Needed_Amount
pickUps_data = pd.read_csv(
    'C:/Users/Toshiba/Documents/OVGU/Thesis/Knime WorkFlow/Dataset/milkrun.csv',
    index_col="ID",
    parse_dates=["Open_From", "Open_To"]
)

# Plot the locations on the map with more info in the ToolTip
for location in pickUps_data.itertuples():
    tooltip = folium.map.Tooltip("<h4><b>ID {}</b></p><p>Supplies needed: <b>{}</b></p>".format(
        location.Index, location.Needed_Amount
    ))

    folium.Marker(
        location=[location.Lat, location.Lon],
        tooltip=tooltip,
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(location.Index),
            spin=True,
            text_color='red',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(m)
    
# The vehicles are all located in Berlin
depot = [52.46802742886862, 13.370658726622388]
 
folium.Marker(
    location=depot,
    icon=folium.Icon(color="green", icon="flag", prefix='fa'),
    setZIndexOffset=1000
).add_to(m)

m

### Model Deployment
We now determine the capacitated vehicle routing model for the milk collection problem by defining decision variables, constraints, and objective function as required by the mathematical model. 

In [4]:
# Define the set of vehicles
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Vehicle
vehicles = list()
for idx in range(3):
    vehicles.append(
        ors.optimization.Vehicle(
            id=idx,
            start=list(reversed(depot)),
            # end=list(reversed(depot)),
            capacity=[300],
            time_window=[1553241600, 1553284800]  # Fri 8-20:00, expressed in POSIX timestamp
        )
    )
    


# Next define the pick up stations (car parts supplier)
# https://openrouteservice-py.readthedocs.io/en/latest/openrouteservice.html#openrouteservice.optimization.Job
pickUps = list()
for pick in pickUps_data.itertuples():
    pickUps.append(
        ors.optimization.Job(
            id=pick.Index,
            location=[pick.Lon, pick.Lat],
            service=1200,  # Unloading time assumed to be 20 minutes at each site
            amount=[pick.Needed_Amount],
            time_windows=[[
                int(pick.Open_From.timestamp()),  # VROOM expects UNIX timestamp
                int(pick.Open_To.timestamp())
            ]]
        )
    )

## Solve
We can now optimize the model openrouteservice optimization engine.

In [5]:
# Initialize a client and make the request
# Get an API key from https://openrouteservice.org/dev/#/signup
ors_client = ors.Client(key='5b3ce3597851110001cf62484d8e8ffbc6c44b72a171738d08f6193a')  
result = ors_client.optimization(
    jobs=pickUps,
    vehicles=vehicles,
    geometry=True
)

# Adding the output to the map
for color, route in zip(['green', 'red', 'blue', 'brown'], result['routes']):
    decoded = ors.convert.decode_polyline(route['geometry'])  # Route geometry is encoded
    gj = folium.GeoJson(
        name='Vehicle {}'.format(route['vehicle']),
        data={"type": "FeatureCollection", "features": [{"type": "Feature",
                                                         "geometry": decoded,
                                                         "properties": {"color": color}
                                                         }]},
        style_function=lambda x: {"color": x['properties']['color']}
    )
    gj.add_child(folium.Tooltip(
        """<h4>Vehicle {vehicle}</h4>
        <b>Distance</b> {distance} m <br>
        <b>Duration</b> {duration} secs
        """.format(**route)
    ))
    gj.add_to(m)
    
folium.LayerControl().add_to(m)

m

---
## Analysis

We print the optimal distance and travel time of each vehicle and the associated supplies picked up.

Take note supplier 3 is not assigned due to capacity constraints therefore the shipping vehicles should be adjusted accordingly in terms of number or size to accomodate for supplier 3. 

---

In [6]:
# extract relevant fields from the decion variable
extract_fields = ['distance', 'amount', 'duration']
data = [{key: route[key] for key in extract_fields} for route in result['routes']]

vehicles_df = pd.DataFrame(data)
vehicles_df.index.name = 'vehicle'
vehicles_df

,distance,amount,duration
vehicle,,,
0,735384,[269],27859
1,821418,[290],32368
2,795314,[273],30277


In [7]:
# Create a list to display the schedule for all vehicles
suppliers = list()
for route in result['routes']:
    vehicle = list()
    for step in route["steps"]:
        vehicle.append(
            [
                step.get("job", "Depot"),  # Supplier ID
                step["arrival"],  # Arrival time
                step["arrival"] + step.get("service", 0),  # Departure time

            ]
        )
    suppliers.append(vehicle)

### Suppiers assigned to the 1st vehicle
The following table defines the schedule of the first vehicle pick-up tour. For example, Supplier 12 at Dresden is served within the the window period(6-20hrs) and the total vehicle capacity of 300 units is not violated.

In [8]:
#Suppiers assigned to vehicle 0
df_suppliers_0 = pd.DataFrame(suppliers[0], columns=["Supplier ID", "Arrival", "Departure"])
df_suppliers_0['Arrival'] = pd.to_datetime(df_suppliers_0['Arrival'], unit='s')
df_suppliers_0['Departure'] = pd.to_datetime(df_suppliers_0['Departure'], unit='s')
df_suppliers_0

,Supplier ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,1,2019-03-22 08:14:05,2019-03-22 08:34:05
2,2,2019-03-22 11:26:05,2019-03-22 11:46:05
3,5,2019-03-22 12:59:02,2019-03-22 13:19:02
4,9,2019-03-22 15:33:52,2019-03-22 15:53:52
5,15,2019-03-22 16:21:53,2019-03-22 16:41:53
6,10,2019-03-22 17:01:17,2019-03-22 17:21:17
7,14,2019-03-22 17:44:19,2019-03-22 18:04:19


### Suppiers assigned to the 2nd vehicle

The following table defines the schedule of the 2nd vehicle pick-up tour.

In [9]:
#Suppiers assigned to vehicle 1
df_suppliers_1 = pd.DataFrame(suppliers[1], columns=["Supplier ID", "Arrival", "Departure"])
df_suppliers_1['Arrival'] = pd.to_datetime(df_suppliers_1['Arrival'], unit='s')
df_suppliers_1['Departure'] = pd.to_datetime(df_suppliers_1['Departure'], unit='s')
df_suppliers_1

,Supplier ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,12,2019-03-22 10:41:55,2019-03-22 11:01:55
2,17,2019-03-22 12:23:54,2019-03-22 12:43:54
3,16,2019-03-22 14:09:30,2019-03-22 14:29:30
4,6,2019-03-22 15:06:42,2019-03-22 15:26:42
5,3,2019-03-22 16:02:34,2019-03-22 16:22:34
6,18,2019-03-22 16:56:12,2019-03-22 17:16:12
7,4,2019-03-22 18:59:28,2019-03-22 19:19:28


### Suppiers assigned to the 3rd vehicle

The following table defines the schedule of the 3rd vehicle pick-up tour.

In [10]:
#Suppiers assigned to vehicle 2
df_suppliers_2 = pd.DataFrame(suppliers[2], columns=["Supplier ID", "Arrival", "Departure"])
df_suppliers_2['Arrival'] = pd.to_datetime(df_suppliers_2['Arrival'], unit='s')
df_suppliers_2['Departure'] = pd.to_datetime(df_suppliers_2['Departure'], unit='s')
df_suppliers_2

,Supplier ID,Arrival,Departure
0,Depot,2019-03-22 08:00:00,2019-03-22 08:00:00
1,11,2019-03-22 10:03:37,2019-03-22 10:23:37
2,8,2019-03-22 11:45:49,2019-03-22 12:05:49
3,13,2019-03-22 14:47:25,2019-03-22 15:07:25
4,7,2019-03-22 17:24:37,2019-03-22 17:44:37
